In [1]:
-- Read more about this program in the official Elm guide:
-- https://guide.elm-lang.org/architecture/effects/http.html

import Html exposing (..)
import Html.Attributes exposing (..)
import Html.Events exposing (..)
import Http
import Json.Decode exposing (int, string, float, Decoder, map3, at, decodeString)
import Debug




main =
  Html.program
    { init = init 1
    , view = view
    , update = update
    , subscriptions = subscriptions
    }



-- MODEL


type alias Crypto =
  { id : Int
  , name : String
  , symbol : String
  }


init : Int -> (Crypto, Cmd Msg)
init id =
  ( Crypto id "" ""
  , getCrypto id
  )



-- UPDATE


type Msg
  = NewCrypto (Result Http.Error String)


update : Msg -> Crypto -> (Crypto, Cmd Msg)
update msg crypto =
  case msg of

    NewCrypto (Ok json) ->
      case decodeString decodeCrypto json of
        Ok newCrypto ->
          ( newCrypto, Cmd.none )
        Err error ->
          ( crypto, Cmd.none )

    NewCrypto (Err _) ->
      (crypto, Cmd.none)



-- VIEW


view : Crypto -> Html Msg
view crypto =
  div []
    [ h2 [] [text (toString crypto.id)]
    , br [] []
    , p [] [text crypto.symbol]
    , p [] [text crypto.name]
    ]



-- SUBSCRIPTIONS


subscriptions : Crypto -> Sub Msg
subscriptions crypto =
  Sub.none



-- HTTP


getCrypto : Int -> Cmd Msg
getCrypto id =
  let
    url = "https://api.coinmarketcap.com/v2/ticker/" ++ (toString id) ++ "/"
  in
    Http.send NewCrypto (Http.getString url)

decodeCrypto : Decoder Crypto
decodeCrypto = map3 Crypto
  (at ["data", "id"] int)
  (at ["data", "name"] string)
  (at ["data", "symbol"] string)

--compile-code